# Passo 1: Baixar Dados

Aqui usaremos uma planilha do Microsoft Excel com dados de compras de usuários em um sistema de varejo. O nome do arquivo é `retail.xlsx`.

In [210]:
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
23.7MB [00:00, 75.5MB/s]


Usaremos a biblioteca `pandas` para fazer a leitura dos dados

In [211]:
import pandas as pd

Aqui fazemos a leitura dos dados para um objeto `DataFrame` do `pandas`.

In [212]:
# Esta linha de código pode demorar cerca de 1 min para rodar
df = pd.read_excel('retail.xlsx')

Agora podemos dar uma observada nos dados para entender como estão organizados.

In [213]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


A tabela acima mostra a estrutura da planilha. Podemos observar os campos:

- **InvoiceNo**: Este é um identificador único para cada compra.
- **StockCode**: Identificador único para cada produto.
- **Description**: Descrição do produto.
- **Quantidade**: Quantidade daquele produto, naquela compra.
- **InvoiceDate**: Dia e hora da compra.
- **CustomerID**: Identificador único do cliente.

In [214]:
df.shape

(541909, 8)

Acima vemos que essa base de dados possui 541.909 registros. Cada registro representa uma linha, ou seja, um produto comprado (cuja quantidade pode ser maior que 1). Diferentes linhas podem representar diferentes itens de uma mesma compra.

# Passo 2: Remoção de Dados Nulos

Neste passo vamos remover da base de dados os dados relativos a compras de produtos onde algum dos dados de interesse estejam faltando.

Começamos observando a quantidade de dados nulos

In [215]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Acima podemos observar há 1.454 registros sem a descrição do produto, e 135.080 registros sem a identificação do cliente comprador.

No código abaixo descartaremos esses dados que aparecem com algum campo nulo.

In [216]:
# Remove dados nulos
df.dropna(inplace=True)
# Verifica novamente
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

# Passo 3: Dicionário de Produtos

In [217]:
# Separa apenas as colunas de código de produto e descrição
products = df[["StockCode", "Description"]]

# Remove duplicados
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [218]:
# Teste do dicionário
#products_dict["84406B"][0]

products_string_dict = {}
for key in products_dict:
  #print(key, '\t', type(key))
  products_string_dict[str(key)] = products_dict[key]

for key in products_string_dict:
  print(key, '\t', type(key))

10002 	 <class 'str'>
10080 	 <class 'str'>
10120 	 <class 'str'>
10125 	 <class 'str'>
10133 	 <class 'str'>
10135 	 <class 'str'>
11001 	 <class 'str'>
15030 	 <class 'str'>
15034 	 <class 'str'>
15036 	 <class 'str'>
15039 	 <class 'str'>
16008 	 <class 'str'>
16010 	 <class 'str'>
16011 	 <class 'str'>
16012 	 <class 'str'>
16014 	 <class 'str'>
16015 	 <class 'str'>
16016 	 <class 'str'>
16033 	 <class 'str'>
16043 	 <class 'str'>
16045 	 <class 'str'>
16046 	 <class 'str'>
16048 	 <class 'str'>
16049 	 <class 'str'>
16052 	 <class 'str'>
16054 	 <class 'str'>
16216 	 <class 'str'>
16218 	 <class 'str'>
16219 	 <class 'str'>
16225 	 <class 'str'>
16235 	 <class 'str'>
16236 	 <class 'str'>
16237 	 <class 'str'>
16238 	 <class 'str'>
16254 	 <class 'str'>
16259 	 <class 'str'>
17001 	 <class 'str'>
17003 	 <class 'str'>
17021 	 <class 'str'>
17038 	 <class 'str'>
17096 	 <class 'str'>
17174 	 <class 'str'>
18007 	 <class 'str'>
20615 	 <class 'str'>
20616 	 <class 'str'>
20617 	 <c

# Passo 4: Preparação dos Dados

Neste passo vamos preprarar os dados transformando o histórico de compras de cada consumidor numa espécie de "frase", onde cada "palavra" é um produto comprado.

Começamos convertendo os códigos de produto (coluna **StockCode**) para string, para usar como "palavras" no treinamento de um modelo word2vec no `gensim` mais tarde.

In [219]:
df['StockCode'] = df['StockCode'].astype(str)

Agora vamos confirmar quantos clientes únicos temos nessa base de dados

In [220]:
# A linha abaixo cria uma lista coletando os ids
# da coluna CustomerID, selecionando apenas ids
# únicos (não recolhe ids repetidos)
customers = df["CustomerID"].unique().tolist()
len(customers)

4372

Acima verificamos que há 4.372 clientes na base de dados. Para cada um desses clientes vamos verificar o histórico de compras, criando 4.372 sequências de compras.

No código abaixo embaralhamos a ordem dos ids dos clientes na lista `customers`.

In [221]:
import random
random.shuffle(customers)

Agora vamos separar dados de treinamento e de validação

In [222]:
# Calcula a quantidade de clientes que usaremos
# para treinamento
train_size = int(0.9 * len(customers))

# Separa os consumidores em duas listas: uma para
# treinamento e a outra para validação
customers_train = customers[:train_size]
customers_val = customers[train_size:len(customers)]

# Baseados nessa separação acima, separamos os dados
df_train = df[df['CustomerID'].isin(customers_train)]
df_val = df[df['CustomerID'].isin(customers_val)]

In [322]:
train_size

3934

Com essas listas de clientes criamos abaixo as sequências de compras de acordo com os históricos de cada cliente.

In [224]:
# Esse módulo serve para mostrar uma barra de progresso
def compile_orders(customers, df):
  ''' Essa função coleta todas compras do histódico
      de cada cliente. O parâmetro customers é a lista
      de ids de clientes e o parâmetro df é o objeto
      DataFrame do pandas com os dados de cada compra.
      O valor retornado é uma lista de listas, onde cada
      lista interna contém a sequência de códigos de produto
      de cada compra, na ordem que se apresentava no
      histórico.
  '''
  orders = []
  for customer in customers:
    order = df[df['CustomerID'] == customer]['StockCode'].tolist()
    orders.append(order)
  return orders

In [225]:
# Aqui separamos as listas de listas de compras. Este código
# demora cerca de 1 minuto para rodar
orders_train = compile_orders(customers_train, df_train)
orders_val = compile_orders(customers_val, df_val)

# Exercício

In [226]:
# Language Model Training
## Training samples (window of size N)
## Window size of N: N-1 first elements are FEATURES (input) and the last one is the LABEL (output) 
#                           |
#                           |__\ This is a SAMPLE in the dataset which can later use to train a LANGUAGE MODEL
#                              /

# OBS.:
# a common way to calculate a similarity score when dealing with vectors is COSINE_SIMILARITY function
# N-grams is commonly used to train language models

# dataset generation

# CONITNUOUS BAG OF WORDS architecture
# SKIPGRAM architecture
# |_ add more SAMPLES for each sliding window position

## (1) Treinar um embedding word2vec usando os dados dos clientes da base de dados de treinamento

In [227]:
import gensim

# Vector of samples:
# orders_train

# MAJOR TRAINING PARAMETERS:
# min_count, size, workers

# MIN_COUNT
# Pruning the internal dictionary. 
# Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage.
# In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them
# default value of min_count=5

# SIZE
# is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.
# default value of size=100

# WORKERS
# is for training parallelization, to speed up training.
# default value of workers=3

# ITER
# Number of iterations (epochs) over the corpus
#

language_model = gensim.models.Word2Vec(orders_train, \
                                        min_count=10, \
                                        size=100, \
                                        workers=10, \
                                        iter=10)

# language_model.save("word2vec.model")
# new_model = gensim.models.Word2Vec.load()

#print(len(language_model.wv.vocab))

## (2) Criar uma função onde, fornecendo um código de produto, a função deve:
a. Imprimir na tela a descrição do produto;<br>
b. Buscar os códigos dos produtos mais similares no embedding;<br>
c. Imprimir na tela uma lista com as descrições dos produtos considerados
similares, e suas respectivas taxas de similaridade.

In [228]:
import warnings
warnings.filterwarnings("ignore")

In [305]:
class Model:
  def __init__(self, dictionary, model):
    self.dictionary = dictionary
    self.model = model

  def product_exists(self, productCode):
    if productCode in self.model.wv:
      return True
    else:
      print('Produto não existente na lista de produtos (vocabulário)')
      return False

  def get_product_description(self, productCode):
    if self.product_exists(productCode):
      print(self.dictionary[productCode][0])

  def get_most_similar(self, productCode, topn=5):
    if self.product_exists(productCode):
      products = self.model.most_similar(positive=[productCode], topn=topn)
      print(80*'=')
      print("CODE\tDESCRIPTION \t\t\t\tSIMILARITY")
      print(80*'=')
      for code, similarity in products:
        if self.product_exists(code):
          print(str(code) + "\t" + self.dictionary[code][0] + "\t\t" + str(similarity))
      print("\n")

In [306]:
product = ['22111', '84029G', '71053', '85123A', 'NN']
#a
model = Model(products_string_dict, language_model)

print(model.product_exists(product[0]))
print(model.product_exists(product[1]))
print(model.product_exists(product[2]))
print(model.product_exists(product[3]))
print(model.product_exists(product[4]))

print()

model.get_product_description(product[0])
model.get_product_description(product[1])
model.get_product_description(product[2])
model.get_product_description(product[3])
model.get_product_description(product[4])

print()

#b c
model.get_most_similar(product[0])
model.get_most_similar(product[1])
model.get_most_similar(product[2])
model.get_most_similar(product[3])
model.get_most_similar(product[4])

True
True
True
True
Produto não existente na lista de produtos (vocabulário)
False

SCOTTIE DOG HOT WATER BOTTLE
KNITTED UNION FLAG HOT WATER BOTTLE
WHITE MOROCCAN METAL LANTERN
CREAM HANGING HEART T-LIGHT HOLDER
Produto não existente na lista de produtos (vocabulário)

CODE	DESCRIPTION 				SIMILARITY
84030E	ENGLISH ROSE HOT WATER BOTTLE		0.9422145485877991
22113	GREY HEART HOT WATER BOTTLE		0.9391776323318481
21484	CHICK GREY HOT WATER BOTTLE		0.9297541975975037
21485	RETROSPOT HEART HOT WATER BOTTLE		0.9258362650871277
84029E	RED WOOLLY HOTTIE WHITE HEART.		0.905028760433197


CODE	DESCRIPTION 				SIMILARITY
84029E	RED WOOLLY HOTTIE WHITE HEART.		0.9228936433792114
22111	SCOTTIE DOG HOT WATER BOTTLE		0.867754340171814
21479	WHITE SKULL HOT WATER BOTTLE 		0.863370954990387
84030E	ENGLISH ROSE HOT WATER BOTTLE		0.8534555435180664
22112	CHOCOLATE HOT WATER BOTTLE		0.8526349663734436


CODE	DESCRIPTION 				SIMILARITY
22464	HANGING METAL HEART LANTERN		0.8543045520782471
22784	LANTERN CRE

##(3) Criar outra função onde, fornecendo uma lista completa de compras (lista de produtos comprados por um cliente), a função deve:<br>
**a.** Imprimir na tela uma lista com as descrições dos produtos comprados
fornecidos à função como argumento<br><br>
**b.** Buscar os vetores de embedding de cada produto fornecido na lista, e calcular um vetor médio, como sendo a média desses vetores:
<br>
<br>
**c.** Usando o vetor médio resultante da conta acima, buscar os produtos
considerados similares no embedding. Exclua das sugestões os produtos
que já constam na lista original.
(Dica: Para realizar uma busca usando um vetor como entrada, utilize o método model.similar_by_vector(v) do modelo word2vec do gensim. Note que talvez nem todos os produtos constem no “vocabulário” treinado. Você pode testar isso tentando verificar se o id do
produto consta no vocabulário com código como “if productid in model.wv:”.)<br><br>
d. Imprimir na tela uma lista com as descrições dos produtos sugeridos
como similares, e suas respectivas taxas de similaridade

In [572]:
import numpy as np
#a
def shopListByCustomer(shop_list_by_customer, customer_id):
  print('Itens comprados pelo cliente ' + str(customer_id) + ': ')
  print(37*'=')
  product_codes = shop_list_by_customer.loc[:,['StockCode']]
  product_description = shop_list_by_customer.loc[:,['Description']]
  print(product_description)

  arrays_average = np.zeros(100)

  for index, row in product_codes.iterrows():
    
    if row[0] in language_model.wv:
      try:
        array_components = language_model.wv[int(row[0])] #np.ndarray
      except:
        array_components = language_model.wv[str(row[0])] #np.ndarray
    else:
      print('Produto ' + str(row[0]) + ' não existente na lista de produtos (vocabulário)')
    
    arrays_average += array_components
  
  arrays_average = arrays_average / len(product_codes)
  #print(arrays_average)
  similar_products = language_model.similar_by_vector(arrays_average)
  print('\n' + 19*'=')
  print("PRODUTOS SIMILARES: ")
  print(19*'=')
  #print(similar_products)
  print(80*'=')
  print("CODE\tDESCRIPTION \t\t\t\tSIMILARITY")
  print(80*'=')
  for product in similar_products:
    if product[0] not in product_codes:
      print(product[0] + "\t" + products_string_dict[product[0]][0] + "\t\t" + str(product[1]))


In [573]:
index = 0

shop_list_by_customer = df_train.loc[df_train['CustomerID'] == customers[index]]

shopListByCustomer(shop_list_by_customer, customers[index])

Itens comprados pelo cliente 17769.0: 
                               Description
67570      12 MESSAGE CARDS WITH ENVELOPES
67571   MINI WOODEN HAPPY BIRTHDAY GARLAND
67572          4 TRADITIONAL SPINNING TOPS
67573    S/6 WOODEN SKITTLES IN COTTON BAG
67574            SET OF 6 SOLDIER SKITTLES
...                                    ...
502990          POPPY'S PLAYHOUSE BEDROOM 
502991       POPPY'S PLAYHOUSE LIVINGROOM 
502992           POPPY'S PLAYHOUSE KITCHEN
502993        REX CASH+CARRY JUMBO SHOPPER
502994            ANTIQUE HEART SHELF UNIT

[313 rows x 1 columns]
Produto 47586A não existente na lista de produtos (vocabulário)

PRODUTOS SIMILARES: 
CODE	DESCRIPTION 				SIMILARITY
84077	WORLD WAR 2 GLIDERS ASSTD DESIGNS		0.6375435590744019
23229	VINTAGE DONKEY TAIL GAME 		0.6157958507537842
84051	ASS COLOUR GLOWING TIARAS		0.6043492555618286
90099	NECKLACE+BRACELET SET BLUE HIBISCUS		0.5878710150718689
22622	BOX OF VINTAGE ALPHABET BLOCKS		0.5867213010787964
21888	BINGO SET		0.5

##(4) Finalmente, usando a função criada no item (3) acima, faça testes de sugestões tomando como exemplo compras dos clientes do grupo de validação. Faça alguns testes e demonstre os resultados. Lembre que talvez nem todos produtos comprados pelos clientes no grupo de validação constem entre as compras dos clientes no grupo de treinamento. Demonstre o funcionamento do sistema de recomendação para pelo menos 5 compras diferentes do grupo de validação

In [576]:
index = 0

shop_list_by_customer = df_val.loc[df_val['CustomerID'] == customers_val[index]]

shopListByCustomer(shop_list_by_customer, customers_val[index])

Itens comprados pelo cliente 14354.0: 
                                Description
452801  SET OF 2 CHRISTMAS DECOUPAGE CANDLE
452802        ROMANTIC IMAGES GIFT WRAP SET
452803       CHRISTMAS RETROSPOT ANGEL WOOD
452804    SMALL HANGING IVORY/RED WOOD BIRD
452805               6 RIBBONS RUSTIC CHARM
452806   SET OF 6 RIBBONS VINTAGE CHRISTMAS
522383          T-LIGHT HOLDER HANGING LACE
522384       GROW YOUR OWN FLOWERS SET OF 3
522385         CUPID DESIGN SCENTED CANDLES

PRODUTOS SIMILARES: 
CODE	DESCRIPTION 				SIMILARITY
21807	WHITE CHRISTMAS STAR DECORATION		0.8373386263847351
22075	6 RIBBONS ELEGANT CHRISTMAS 		0.8304757475852966
21801	CHRISTMAS TREE DECORATION WITH BELL		0.8208733797073364
21802	CHRISTMAS TREE HEART DECORATION		0.8131743669509888
22731	3D CHRISTMAS STAMPS STICKERS 		0.8087295293807983
22733	3D TRADITIONAL CHRISTMAS STICKERS		0.8051475882530212
35954	SMALL FOLKART STAR CHRISTMAS DEC		0.8037689328193665
21803	CHRISTMAS TREE STAR DECORATION		0.8016600608825684
23

In [583]:
from random import random

for i in range(5):
  index = int(random() * len(customers_val))
  shop_list_by_customer = df_val.loc[df_val['CustomerID'] == customers_val[index]]
  shopListByCustomer(shop_list_by_customer, customers_val[index])
  print("\n\n\n")

Itens comprados pelo cliente 15705.0: 
                            Description
200241  VICTORIAN GLASS HANGING T-LIGHT
200242            DOORMAT ENGLISH ROSE 
200243            DOORMAT RED RETROSPOT
200244               DOORMAT UNION FLAG
200245       DOORMAT MULTICOLOUR STRIPE
200246                    FIRST AID TIN

PRODUTOS SIMILARES: 
CODE	DESCRIPTION 				SIMILARITY
48184	DOORMAT ENGLISH ROSE 		0.9297249913215637
21524	DOORMAT SPOTTY HOME SWEET HOME		0.9146657586097717
48116	DOORMAT MULTICOLOUR STRIPE		0.9137594699859619
21523	DOORMAT FANCY FONT HOME SWEET HOME		0.9070110321044922
48138	DOORMAT UNION FLAG		0.9042071104049683
22366	DOORMAT AIRMAIL 		0.8975205421447754
22692	DOORMAT WELCOME TO OUR HOME		0.8930812478065491
48188	DOORMAT WELCOME PUPPIES		0.8901119232177734
48187	DOORMAT NEW ENGLAND		0.8876519203186035
48111	DOORMAT 3 SMILEY CATS		0.8875356912612915




Itens comprados pelo cliente 16114.0: 
                                Description
236034      GREEN REGENCY TEACUP AN